# Assignment

In this assignment, we will create an embedding of the characters in the novel "Les Miserables" by Victor Hugo.

Complete the following tasks and upload your notebook to your GitHub repository.

1. Fill in the black functions, marked by, "\#TODO", in the notebook
2. Update this notebook by using `git add`, `git commit`, and then `git push`.
3. The notebook will be automatically graded, and your score will be shown on GitHub. See [how to check the results on GitHub](https://docs.github.com/en/education/manage-coursework-with-github-classroom/learn-with-github-classroom/view-autograding-results)

Dont's:
- Do not import any libraries except for the ones that are already imported. The grading script will not work if you import additional libraries.
- Do not change the name of the functions. The grading script will not work if you change the name of the functions.
- DO not remove the cells with tag "gradable". If you happen to remove the cells with tag "gradable", recreate the cell and add the tag "gradable" to the cell.

Do's:
- Make the code clearn and easy to understand. Consider using linters & formatters such as `black`. [Youtube: Linters and fixers: never worry about code formatting again (Vim + Ale + Flake8 & Black for Python) by Yong-Yeol Ahn](https://www.youtube.com/watch?v=4FKPySR6HLk)
- You can add additional cells as you like. If you want to include the additional cells in the grading, you need to add the tag "gradable" to the cell. Otherwise, your additional cells will be stripped out when grading. 

# Preparation

The data we will work with is stored in the file `data` folder. It has two files: 
1. `chapter-character.csv`: A table that contains the chapter number and the characters in the chapter.
2. `character.csv`: A table that contains the characters and their descriptions.

The `chapter-character.csv` has the following columns:
- `chapter`: The chapter number
- `character`: The character names

The `character.csv` has the following columns:
- `abbreviation`: The abbreviation of the character
- `description`: The description of the character
- `name`: The full name of the character


The size of the networks:

In [ ]:
#TODO
def func(g):
    """
    TODO: Write a function that returns the number of nodes and edges in the graph.
    """
    pass